In [1]:
import numpy as np
import random
import gensim
import pandas as pd
from owlready2 import *

Load ontology and embeddings

In [2]:
#OntoGeoLógica populada (OntoGeoLogicaPopulada.owl)
onto = get_ontology("../../KnowledgeGraph/OntoGeoLogicaInstanciasRelacoes.owl")
onto.load()

get_ontology("http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2#")

In [3]:
# Modelo OWL2Vec - 
PetroOntoVec = gensim.models.Word2Vec.load("../../Embeddings/PetroOntoVec/PetroOntoVec_plus_PetroVec_OeG-NP2-V2/outputontology.embeddings")

### Carregando dataset

In [4]:
# Listas de treino
text_treino = np.load('../PetroNER-LinkedEntity/treino - sentences.npy')
entities_treino = np.load('../PetroNER-LinkedEntity/treino - entities.npy')
classes_treino = np.load('../PetroNER-LinkedEntity/treino - classes.npy')
URI_treino = np.load('../PetroNER-LinkedEntity/treino - URI.npy', allow_pickle=True)
#URIvec_treino = np.load('../../Corpora/PetroNER-LinkedEntity/treino - URI_vectors.npy')

# Listas de validação
text_valid = np.load('../PetroNER-LinkedEntity/valid - sentences.npy')
entities_valid = np.load('../PetroNER-LinkedEntity/valid - entities.npy')
classes_valid = np.load('../PetroNER-LinkedEntity/valid - classes.npy')
URI_valid = np.load('../PetroNER-LinkedEntity/valid - URI.npy', allow_pickle=True)
#URIvec_valid = np.load('../../Corpora/PetroNER-LinkedEntity/valid - URI_vectors.npy')

# Listas de teste
text_teste = np.load('../PetroNER-LinkedEntity/teste - sentences.npy')
entities_teste = np.load('../PetroNER-LinkedEntity/teste - entities.npy')
classes_teste = np.load('../PetroNER-LinkedEntity/teste - classes.npy')
URI_teste = np.load('../PetroNER-LinkedEntity/teste - URI.npy', allow_pickle=True)
#URIvec_teste = np.load('../../Corpora/PetroNER-LinkedEntity/teste - URI_vectors.npy')

In [5]:
print ('Treino')
print('Texto: ', len(text_treino))
print('Entidades: ', len(entities_treino))
print('Classes: ', len(classes_treino))
print('URI: ', len(URI_treino))
#print('URIvec: :', len(URIvec_treino))

print ('\n Validação')
print('Texto: ', len(text_valid))
print('Entidades: ', len(entities_valid))
print('Classes: ', len(classes_valid))
print('URI: ', len(URI_valid))
#print('URIvec: :', len(URIvec_valid))

print ('\n Teste')
print('Texto: ', len(text_teste))
print('Entidades: ', len(entities_teste))
print('Classes: ', len(classes_teste))
print('URI: ', len(URI_teste))
#print('URIvec: :', len(URIvec_teste))

Treino
Texto:  14277
Entidades:  14277
Classes:  14277
URI:  14277

 Validação
Texto:  1785
Entidades:  1785
Classes:  1785
URI:  1785

 Teste
Texto:  3216
Entidades:  3216
Classes:  3216
URI:  3216


### Buscando vetores do PetroOntoVec para cada URI

In [6]:
def buscando_URI_vec(text, entities, classes_NER, ENT_URI):

    # Separando as listas de sentenças, entidades, classes, URI e vetores
    text_new = []
    entities_new = []
    classes_NER_new = [] 
    ENT_URI_new = []
    URI_Vec_new = []

    URI_ausente_Onto = []

    n = 0 
    #m = 0
    #Iterando por todas as sentenças, desconsiderando aquelas cujas entidades não possuem URI
    for i in range(len(text)):
        n = n + 1
        if ENT_URI[i] != None:
            # Quebrando as entidades que possuem mais de um URI (ex: "#Aptian,#Miocene" e "#Cenozoic,#Mesozoic")
            ENT_URI_n = ENT_URI[i].split(',#')

            # Iterando pelas ENT_URI após serem quebradas e acrescentando '#" naquelas que perderam na operação de split 
            for ent in ENT_URI_n:
                if ent[0] != '#':
                    ent = '#' + ent
                try:
                    #Acrescentar a URI da entidade e o vetor da URI nas listas
                    URI_Vec_new.append(PetroOntoVec['http://www.semanticweb.org/bg40/ontologies/2022/5/untitled-ontology-2' + ent])
                    ENT_URI_new.append(ent)

                    # O texto, a entidade no texto e a classe são repetidas quando existem multiplas URI para uma mesma entidade
                    text_new.append(text[i])
                    entities_new.append(entities[i])
                    classes_NER_new.append(classes_NER[i])
                except:
                    #print("URI presente no arquivo CONLLU mas ausente na Ontologia: ", ent)
                    URI_ausente_Onto.append(ent)

    # Transformando as listas em Numpy array
    text = np.array(text_new)
    entities = np.array(entities_new)
    classes_NER = np.array(classes_NER_new) 
    ENT_URI = np.array(ENT_URI_new)
    URI_Vec = np.array(URI_Vec_new)
    URI_ausente_Onto = np.array(URI_ausente_Onto)

    print('Total de sentenças anotadas: ', n)
    print('N° de sentenças: ', len(text))
    print ('N° de entidades: ', len(entities))
    print('N° de classes: ', len(classes_NER))
    print('N° de entidades_ID: ', len(ENT_URI))       
    print('N° de vetores de URI: ', len(URI_Vec))       
    
    return(text, entities, classes_NER, ENT_URI, URI_Vec, URI_ausente_Onto)

In [7]:
#Treino
(text_treino,
 entities_treino,
 classes_treino,
 URI_treino,
 URIvec_treino,
 URI_ausente_Onto_treino) = buscando_URI_vec(text_treino,
                                             entities_treino,
                                             classes_treino,
                                             URI_treino)

#Validação
(text_valid,
 entities_valid,
 classes_valid,
 URI_valid,
 URIvec_valid,
 URI_ausente_Onto_valid) = buscando_URI_vec(text_valid,
                                            entities_valid,
                                            classes_valid,
                                            URI_valid)

#Teste
(text_teste,
 entities_teste,
 classes_teste,
 URI_teste,
 URIvec_teste,
 URI_ausente_Onto_teste) = buscando_URI_vec(text_teste,
                                            entities_teste,
                                            classes_teste,
                                            URI_teste)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:27: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


Total de sentenças anotadas:  14277
N° de sentenças:  13434
N° de entidades:  13434
N° de classes:  13434
N° de entidades_ID:  13434
N° de vetores de URI:  13434
Total de sentenças anotadas:  1785
N° de sentenças:  1696
N° de entidades:  1696
N° de classes:  1696
N° de entidades_ID:  1696
N° de vetores de URI:  1696
Total de sentenças anotadas:  3216
N° de sentenças:  2869
N° de entidades:  2869
N° de classes:  2869
N° de entidades_ID:  2869
N° de vetores de URI:  2869


In [8]:
print('URI no PetroNER_Treino ausentes da ontologia: ')
print(set(URI_ausente_Onto_treino))
print('')
print('URI no PetroNER_Validação ausentes da ontologia: ')
print(set(URI_ausente_Onto_valid))
print('')
print('URI no PetroNER_Teste ausentes da ontologia: ')
print(set(URI_ausente_Onto_teste))

URI no PetroNER_Treino ausentes da ontologia: 
{'#inter_membro_013', '#INTER_BACIA_007', '#INTER_BACIA_014', '#inter_membro_008', '#INTER_BACIA_010', '#inter_formacao_023', '#EXTERNO_EST_FISICA_ROCHA_002', '#externo_textura_005', '#calcilutite', '#INTER_BACIA_020', '#INTER_BACIA_004', '#??', '#EXTERNO_EST_FISICA_ROCHA_003', '#inter_membro_009', '#inter_membro_012', '#EXTERNO_EST_FISICA_ROCHA_005', '#externo_formacao_003', '#inter_formacao_017', '#externo_textura_002', '#externo_grupo_003', '#POCO_GENERICO', '#INTER_BACIA_024', '#inter_grupo_002', '#inter_formacao_024', '#INTER_BACIA_017', '#inter_membro_011', '#inter_grupo_001', '#INTER_BACIA_003', '#inter_formacao_021', '#grupo_05', '#INTER_CAMPO_0001', '#externo_formacao_001', '#inter_formacao_027', '#INTER_BACIA_025', '#INTER_BACIA_023', '#INTER_BACIA_022', '#supergrupo_001', '#INTER_BACIA_019', '#inter_formacao_019', '#INTER_BACIA_011', '#externo_grupo_002', '#EXTERNO_EST_FISICA_ROCHA_007', '#inter_membro_010', '#inter_formacao_020

### Preparando dataset para treinar rede Siamesa

In [9]:
#Dataframe com as URI, sentenças e classes

def dataset_siamesa(classes, URI, URIvec, text, x_easy, x_hard):
       
    df = pd.DataFrame({'Classe': classes, 'URI': URI, 'Sentença': text}) #, 'URIvec': URIvec})
    
    # Arrays para armazenar as sentenças âncoras, positivas e negativas 
    URI_anchor = np.array([])
    URIvec_anchor = np.array([])
    dataset_anchor = np.array([])
    dataset_positive = np.array([])
    dataset_negative = np.array([])

    #Quantidade de exemplos negativos fáceis e difíceis
    #x_easy = 20
    #x_hard = 30

    # iterando por todas as URI
    for i in set(URI):

        try:
            
            # Subgrupo apenas com a URI selecionada
            df_URI = df[df['URI'] == i]

            # Classe da URI selecionada
            classe_da_URI = df_URI['Classe'].values[0]
            # Vetor da URI selecionada
            vetor_da_URI = URIvec[df_URI.index[0]]
            # Subgrupo com sentenças negativa 'fácil' (URI diferente e classe diferente).
            df_negative_easy = df[(df['URI'] != i) & (df['Classe'] != classe_da_URI)]
            # Subgrupo com sentenças negativa 'difícil' (URI diferente mas da mesma classe).
            df_negative_hard = df[(df['URI'] != i) & (df['Classe'] == classe_da_URI)]

            # Pular os casos onde só existe uma sentença para a URI selecionada
            if (len(df_URI) > 1) & (len(df_negative_easy) > 1) & (len(df_negative_hard) > 1):

                ### Construindo triplets com exemplos negativos fáceis

                # Selecionando as sentenças âncoras 
                # (verificamos o número de combinações possíveis entre sentenças com mesma URI "len(df_URI)*(len(df_URI) - 1)".
                #  Se X_easy for maior que esse valor, amostramos apenas essa quantidade de amostras.
                df_ancora = df_URI.sample(min(len(df_URI)*(len(df_URI) - 1) , x_easy), replace=True)

                # itera pelas senteças âncoras
                for row in df_ancora.iterrows():

                    anchor = row[1]['Sentença']
                    # Seleciona uma exemplo positivo diferente da sentença âncora
                    positive = df_URI[df_URI['Sentença'] != anchor].sample(1)['Sentença'].values[0]

                    # Seleciona uma sentença negativa
                    negative_easy = df_negative_easy.sample(1)['Sentença'].values[0]

                    # Concatena com as arrays que armazenam as senteças e URI
                    URI_anchor = np.concatenate((URI_anchor, np.array([np.str_(i)])), axis=0)
                    URIvec_anchor = np.concatenate((URIvec_anchor, vetor_da_URI), axis=0)
                    dataset_anchor = np.concatenate((dataset_anchor, np.array([anchor])), axis=0)
                    #dataset_anchor  = np.concatenate((dataset_anchor, anchor), axis=0)
                    dataset_positive = np.concatenate((dataset_positive, np.array([positive])), axis=0)
                    dataset_negative = np.concatenate((dataset_negative, np.array([negative_easy])), axis=0)

                ### Construindo triplets com exemplos negativos difíceis

                # Selecionando as sentenças âncoras 
                # (verificamos o número de combinações possíveis entre sentenças com mesma URI "len(df_URI)*(len(df_URI) - 1)".
                #  Se X_hard for maior que esse valor, amostramos apenas essa quantidade de amostras.
                
                df_ancora = df_URI.sample(min(len(df_URI)*(len(df_URI) - 1), x_hard), replace=True)

                # itera pelas senteças âncoras
                for row in df_ancora.iterrows():
                    anchor = row[1]['Sentença']

                    # Seleciona uma exemplo positivo diferente da sentença âncora
                    positive = df_URI[df_URI['Sentença'] != anchor].sample(1)['Sentença'].values[0]

                    # Seleciona uma sentença negativa
                    negative_hard = df_negative_hard.sample(1)['Sentença'].values[0]

                    # Concatena com as arrays que armazenam as senteças
                    URI_anchor = np.concatenate((URI_anchor, np.array([np.str_(i)])), axis=1)
                    URIvec_anchor = np.concatenate((URIvec_anchor, vetor_da_URI), axis=0)
                    dataset_anchor  = np.concatenate((dataset_anchor, np.array([anchor])), axis=0)
                    #dataset_anchor  = np.concatenate((dataset_anchor, anchor), axis=0)
                    dataset_positive  = np.concatenate((dataset_positive, np.array([positive])), axis=0)
                    dataset_negative  = np.concatenate((dataset_negative, np.array([negative_hard])), axis=0)

        except:
            pass
    
    #Ajustando o formato da array URIvec_anchor
    URIvec_anchor = np.reshape(URIvec_anchor, (-1, len(URIvec[0])))
    
    return(URI_anchor, URIvec_anchor, dataset_anchor, dataset_positive, dataset_negative)

In [10]:
(URI_anchor_treino,
 URIvec_anchor_treino,
 dataset_anchor_treino,
 dataset_positive_treino,
 dataset_negative_treino) = dataset_siamesa(classes_treino,
                                            URI_treino,
                                            URIvec_treino,
                                            text_treino, 50, 50)

(URI_anchor_valid, 
 URIvec_anchor_valid,
 dataset_anchor_valid,
 dataset_positive_valid,
 dataset_negative_valid) = dataset_siamesa(classes_valid,
                                           URI_valid,
                                           URIvec_valid,
                                           text_valid, 50, 50)

(URI_anchor_teste,
 URIvec_anchor_teste,
 dataset_anchor_teste,
 dataset_positive_teste,
 dataset_negative_teste) = dataset_siamesa(classes_teste,
                                           URI_teste,
                                           URIvec_teste,   
                                           text_teste, 50, 50)

In [11]:
print('Dataset treino: ', len(URI_anchor_treino))
print('Dataset validação: ', len(dataset_anchor_valid))
print('Dataset teste: ', len(dataset_anchor_teste))

Dataset treino:  12408
Dataset validação:  3290
Dataset teste:  4850


In [12]:
n = 51

print(dataset_anchor_treino[n])
print(dataset_positive_treino[n])
print(dataset_negative_treino[n])
print(URI_anchor_treino[n])
print(URIvec_anchor_treino[n])

[ESTRUTURA_FÍSICA] estrutura | Há cloritas que têm sua [E] estrutura [/E] al-terada sob esses valores de temperatura, e há caulinitas muito bem cris-talizadas que se mantêm inalteradas nessas condições( THIRY, 1974; BROWN & BRINDLEY, 1980); b) operacional: acima destes valores de temperatura, a lâmina de vidro usada no laboratório se funde e não se mantém mais na posição vertical, Quando se necessita de temperaturas mais elevadas, usa-se a lâmina na posição horizontal..
[ESTRUTURA_FÍSICA] estrutura | Esta é uma [E] estrutura [/E] com forte resposta aeromagnética e gravimétrica, ao longo da qual, tardiamente, foram intrudidos vários corpos alcalinos e kimberlíticos..
[FLUIDODATERRA_o] águas | Aparentemente, um bom número de pesquisadores acredita numa rápida diminuição da temperatura das [E] águas [/E] de fundo do Oceano..
#geological_structure
[-0.39573058 -0.63598138  1.08965802 -0.98029166 -1.88695598  3.81358504
 -0.15760019 -0.64616621  4.19330072  1.76930308  2.67695022  1.2084559

### Salvando dataset


In [13]:
#Treino
with open('URI_anchor_treino.npy', 'wb') as f:
    np.save(f, URI_anchor_treino)
with open('URIvec_anchor_treino.npy', 'wb') as f:
    np.save(f, URIvec_anchor_treino)
with open('anchor_treino.npy', 'wb') as f:
    np.save(f, dataset_anchor_treino)
with open('positive_treino.npy', 'wb') as f:
    np.save(f, dataset_positive_treino)
with open('negative_treino.npy', 'wb') as f:
    np.save(f, dataset_negative_treino)

    
#Validação
with open('URI_anchor_valid.npy', 'wb') as f:
    np.save(f, URI_anchor_valid)
with open('URIvec_anchor_valid.npy', 'wb') as f:
    np.save(f, URIvec_anchor_valid)
with open('anchor_valid.npy', 'wb') as f:
    np.save(f, dataset_anchor_valid)
with open('positive_valid.npy', 'wb') as f:
    np.save(f, dataset_positive_valid)
with open('negative_valid.npy', 'wb') as f:
    np.save(f, dataset_negative_valid)
    
#Teste
with open('URI_anchor_teste.npy', 'wb') as f:
    np.save(f, URI_anchor_teste)
with open('URIvec_anchor_teste.npy', 'wb') as f:
    np.save(f, URIvec_anchor_teste)
with open('anchor_teste.npy', 'wb') as f:
    np.save(f, dataset_anchor_teste)
with open('positive_teste.npy', 'wb') as f:
    np.save(f, dataset_positive_teste)
with open('negative_teste.npy', 'wb') as f:
    np.save(f, dataset_negative_teste)